In [1]:
from singlequbit import *
import multiprocessing as mp

In [2]:
omega = 1  # qubit level splitting
g = 0.2*omega  # qubit-TLS coupling
gamma = 0.05*omega  # TLS decay rate
N=0 # inverse bath temperature
T = 15 # final time
nt = 100 # number of time steps
lambdas=np.linspace(0.0005,0.005,3)#Number of lambdas to probe

In [3]:
rho_th = qutip.Qobj(qutip.tensor(qutip.Qobj(np.diag([1,0])), qutip.Qobj(np.diag([1,0]))))
rho_q_trg = qutip.Qobj(np.diag([1, 0]))
rho_T_trg = qutip.Qobj(np.diag([0, 1]))
rho_trg = qutip.tensor(rho_q_trg, rho_T_trg)

In [4]:
def torun(i):
    return function_parallel(omega, g, gamma, N, rho_th, rho_trg, nt,i, T)


In [6]:
%%capture
N = mp.cpu_count()
with mp.Pool(processes=N) as p:
    results = p.map(torun, list(lambdas));

Iteration 0
Iteration 0
    objectives:
    objectives:
        1:ρ₀[2⊗2,2⊗2] to ρ₁[2⊗2,2⊗2] via [𝓛₀[[2⊗2,2⊗2],[2⊗2,2⊗2]], [𝓛₁[[2⊗2,2⊗2],[2⊗2,2⊗2]], u₁(t)]]
        1:ρ₀[2⊗2,2⊗2] to ρ₁[2⊗2,2⊗2] via [𝓛₀[[2⊗2,2⊗2],[2⊗2,2⊗2]], [𝓛₁[[2⊗2,2⊗2],[2⊗2,2⊗2]], u₁(t)]]
    adjoint objectives:
    adjoint objectives:
        1:ρ₂[2⊗2,2⊗2] to ρ₃[2⊗2,2⊗2] via [𝓛₂[[2⊗2,2⊗2],[2⊗2,2⊗2]], [𝓛₃[[2⊗2,2⊗2],[2⊗2,2⊗2]], u₁(t)]]
        1:ρ₂[2⊗2,2⊗2] to ρ₃[2⊗2,2⊗2] via [𝓛₂[[2⊗2,2⊗2],[2⊗2,2⊗2]], [𝓛₃[[2⊗2,2⊗2],[2⊗2,2⊗2]], u₁(t)]]
    chi_constructor: chis_qubit
    chi_constructor: chis_qubit
    mu: derivative_wrt_pulse
    mu: derivative_wrt_pulse
    S(t) (ranges): [0.000000, 1.000000]
    S(t) (ranges): [0.000000, 1.000000]
    iter_start: 0
    iter_start: 0
    iter_stop: 100
    iter_stop: 100
    duration: 0.0 secs (started at 2023-07-28 16:19:45)
    duration: 0.0 secs (started at 2023-07-28 16:19:45)
    optimized pulses (ranges): [0.00, 0.00]
    optimized pulses (ranges): [0.00, 0.00]
    ∫gₐ(t)dt: 0.

In [9]:
results.iters

AttributeError: 'list' object has no attribute 'iters'